In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from tpot import TPOTRegressor
from sklearn.metrics import make_scorer
import matplotlib.pyplot as plt
from featuresv2 import features, targets, metadata
%matplotlib inline

## Version 2 Model

In [4]:
data = pd.read_csv('outputs/model_training_data_v2.csv')

In [5]:
round_matchups = [
    ('Sydney Swans','Melbourne'),
    ('Brisbane Lions','Carlton'),
    ('Gold Coast Suns','Richmond'),
    ('GWS Giants','Collingwood')]

In [6]:
round1 = [('Carlton', 'Richmond'),
('Collingwood','Sydney Swans'),
('Essendon','Hawthorn'),
('GWS Giants','North Melbourne'),
('Geelong Cats','St Kilda'),
('Gold Coast Suns','Adelaide Crows'),
('Melbourne','Western Bulldogs'),
('Port Adelaide','West Coast Eagles'),
('Fremantle','Brisbane Lions')]

In [7]:
targets

['homeTeamScore.matchScore.totalScore',
 'awayTeamScore.matchScore.totalScore',
 'score_diff']

In [8]:
def generate_test_train_split(data, features, target, test_size=0.3):
    X = data[features]
    y = data[target]

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    
    return {"x_train":x_train,"x_test":x_test,"y_train":y_train,"y_test":y_test}

In [9]:
def tpot_pipeline_v2(data, features, target, size=20, scoring='r2'):
    data_ = generate_test_train_split(data, features, target)

    my_scorer = make_scorer(mean_squared_error, greater_is_better=False)

    pipeline_optimizer = TPOTRegressor(generations=5, population_size=size, verbosity=2, scoring=scoring)

    pipeline_optimizer.fit(data_['x_train'], data_['y_train'])
    
    print(pipeline_optimizer.score(data_['x_test'], data_['y_test']))
    
    return pipeline_optimizer

In [10]:
def return_stats_inputv2(data, home_team, away_team, model_inputs, games=5):
    '''Take in player_stats and returns aggregated team stats for previous home/away team matchups'''
    
#     if 'team.name' not in model_inputs:
#         model_inputs.append('team.name')
        
    match_ups = data[(data['match.homeTeam.name']==home_team) & (data['match.awayTeam.name']==away_team)]
    stats = match_ups[model_inputs+['match.homeTeam.name']].groupby('match.homeTeam.name').mean()
    
    return stats[model_inputs]

In [11]:
def generate_margin_tear_offv2(model, round_matchups, team_stats, model_inputs):
    print('-------------------' + str(model.fitted_pipeline_[0]).replace('()','') + ' TEAR-OFF-------------------')
    for tup in round_matchups:
        predict = model.predict(return_stats_inputv2(team_stats, tup[0], tup[1], model_inputs))[0]
        if predict < 0:
            winner = tup[1]
        else:
            winner = tup[0]
        print("------------------------------------------------")
        print("Home: " + tup[0])
        print("Away: " + tup[1])
        print("Winner: " + winner + "\t       " + "Margin: " + str(abs(round(predict))))

In [12]:
def generate_score_tear_offv2(home_model, away_model, round_matchups, team_stats, model_inputs):
   # print('-------------------' + str(model.fitted_pipeline_[0]).replace('()','') + ' TEAR-OFF-------------------')
    preds = []
    for tup in round_matchups:
        h_predict = home_model.predict(return_stats_inputv2(team_stats, tup[0], tup[1], model_inputs))[0]
        a_predict = away_model.predict(return_stats_inputv2(team_stats, tup[0], tup[1], model_inputs))[0]
        if h_predict < a_predict:
            winner = tup[1]
            margin = round(a_predict) - round(h_predict)
        else:
            winner = tup[0]
            margin = round(h_predict) - round(a_predict)
        paddingh = 25 - len("Home: " + tup[0])
        paddinga = 25 - len("Away: " + tup[1])
        print("------------------------------------------------")
        print("Home: " + tup[0] + " "*paddingh + str(abs(round(h_predict))))
        print("Away: " + tup[1] + " "*paddinga + str(abs(round(a_predict))))
        print("Winner: " + winner + " by "+ str(margin) + " points")
        print("Total Score: " + str(round(h_predict) + round(a_predict)))
        preds.append((h_predict, a_predict, margin))
    return preds

In [13]:
home_model = tpot_pipeline_v2(data, features, targets[0], size=10)

Optimization Progress:   0%|          | 0/60 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9556685973219192

Generation 2 - Current best internal CV score: 0.9556685973219192

Generation 3 - Current best internal CV score: 0.9556685973219192

Generation 4 - Current best internal CV score: 0.9556685973219192

Generation 5 - Current best internal CV score: 0.9558128993275432

Best pipeline: DecisionTreeRegressor(RidgeCV(input_matrix), max_depth=7, min_samples_leaf=10, min_samples_split=4)
0.9566829424976252


C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(


In [14]:
away_model = tpot_pipeline_v2(data, features, targets[1], size=10)

Optimization Progress:   0%|          | 0/60 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9706503413446876

Generation 2 - Current best internal CV score: 0.9718329328305064

Generation 3 - Current best internal CV score: 0.9718329328305064

Generation 4 - Current best internal CV score: 0.9718857502803662

Generation 5 - Current best internal CV score: 0.9726212080606336

Best pipeline: ElasticNetCV(RobustScaler(RidgeCV(input_matrix)), l1_ratio=0.8, tol=1e-05)
0.9719608127255673


C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(


In [15]:
margin_model = tpot_pipeline_v2(data, features, targets[2], size=10)

Optimization Progress:   0%|          | 0/60 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9731446552616015

Generation 2 - Current best internal CV score: 0.9731446552616015

Generation 3 - Current best internal CV score: 0.9731446552616015

Generation 4 - Current best internal CV score: 0.9731446552616015

Generation 5 - Current best internal CV score: 0.9731446552616015

Best pipeline: RidgeCV(input_matrix)
0.9713170802425867


In [18]:
results = generate_score_tear_offv2(home_model, away_model, round1, data, features)

------------------------------------------------
Home: Carlton            83
Away: Richmond           84
Winner: Richmond by 1 points
Total Score: 167
------------------------------------------------
Home: Collingwood        58
Away: Sydney Swans       47
Winner: Collingwood by 11 points
Total Score: 105
------------------------------------------------
Home: Essendon           100
Away: Hawthorn           82
Winner: Essendon by 18 points
Total Score: 182
------------------------------------------------
Home: GWS Giants         57
Away: North Melbourne    84
Winner: North Melbourne by 27 points
Total Score: 141


C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitte

------------------------------------------------
Home: Geelong Cats       96
Away: St Kilda           67
Winner: Geelong Cats by 29 points
Total Score: 163
------------------------------------------------
Home: Gold Coast Suns    87
Away: Adelaide Crows     86
Winner: Gold Coast Suns by 1 points
Total Score: 173
------------------------------------------------
Home: Melbourne          100
Away: Western Bulldogs   73
Winner: Melbourne by 27 points
Total Score: 173
------------------------------------------------
Home: Port Adelaide      105
Away: West Coast Eagles  45
Winner: Port Adelaide by 60 points
Total Score: 150
------------------------------------------------
Home: Fremantle          76
Away: Brisbane Lions     88
Winner: Brisbane Lions by 12 points
Total Score: 164


C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(


In [17]:
generate_margin_tear_offv2(margin_model, round1, data, features)

-------------------RidgeCV TEAR-OFF-------------------
------------------------------------------------
Home: Carlton
Away: Richmond
Winner: Richmond	       Margin: 4
------------------------------------------------
Home: Collingwood
Away: Sydney Swans
Winner: Collingwood	       Margin: 15
------------------------------------------------
Home: Essendon
Away: Hawthorn
Winner: Essendon	       Margin: 17
------------------------------------------------
Home: GWS Giants
Away: North Melbourne
Winner: North Melbourne	       Margin: 27
------------------------------------------------
Home: Geelong Cats
Away: St Kilda
Winner: Geelong Cats	       Margin: 27
------------------------------------------------
Home: Gold Coast Suns
Away: Adelaide Crows
Winner: Gold Coast Suns	       Margin: 3
------------------------------------------------
Home: Melbourne
Away: Western Bulldogs
Winner: Melbourne	       Margin: 29
------------------------------------------------
Home: Port Adelaide
Away: West Coast 